In [6]:
import pandas as pd 
import numpy as np 

import recordlinker

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
iowa_matches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_matches.csv')
iowa_nonmatches = pd.read_csv('/Users/kailinlu/Desktop/QMSSWork/RecordLinking/recordlinker/recordlinker/data/iowa_nonmatches.csv')

In [4]:
iowa_matches.head()

,uid1915,fname1915,lname1915,fullname1915,yob1915,hhid,fname1940,lname1940,fullname1940,yob1940,uid-hhid
0,uid0910071227,donald d,cutler,donald d cutler,1911,19067,donald dean,cutler,donald dean cutler,1911,uid0910071227-19067
1,uid0063131339,homer,taylor,homer taylor,1912,71505,homer ellis,taylor,homer ellis taylor,1912,uid0063131339-71505
2,uid0044088276,earl,stearnes,earl stearnes,1899,109708,earl,stearns,earl stearns,1900,uid0044088276-109708
3,uid0067053130,theodore,hornaday,theodore hornaday,1904,108304,theodore i,harnaday,theodore i harnaday,1904,uid0067053130-108304
4,uid0066046148,jack r,turner,jack r turner,1907,105092,jack r,turner,jack r turner,1907,uid0066046148-105092


In [86]:
ORIG_LENGTH = 20 
BATCH_SIZE = 32
LATENT_DIM = 16
ENCODE_DIM = [128, 128] 
DECODE_DIM = [128, 128]
LR = 1e-3
EPOCHS=300

namesA = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1915',
                                                     max_length=ORIG_LENGTH, embed_type='letters')
namesB = recordlinker.preprocess.create_training_set(iowa_matches, 'lname1940',
                                                     max_length=ORIG_LENGTH, embed_type='letters')

vae = recordlinker.model.VAE(batch_size=BATCH_SIZE,
                             orig_dim=ORIG_LENGTH, 
                             latent_dim=LATENT_DIM,
                             encode_dim=ENCODE_DIM,
                             decode_dim=DECODE_DIM,
                             lr=LR)
vae.train(namesA, namesB, 
          epochs=EPOCHS, 
          run_id='test_run_1', 
          tensorboard=True, 
          earlystop=False, 
          reconstruct=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (32, 20)             0                                            
__________________________________________________________________________________________________
enc_0 (Dense)                   (32, 128)            2688        input_1[0][0]                    
__________________________________________________________________________________________________
enc_1 (Dense)                   (32, 128)            16512       enc_0[0][0]                      
__________________________________________________________________________________________________
mu (Dense)                      (32, 16)             2064        enc_1[0][0]                      
__________________________________________________________________________________________________
log_sigma 

3456/3456 [==============================] - 0s 66us/step - loss: 4.9960 - acc: 0.7173 - val_loss: 5.2406 - val_acc: 0.7106
Epoch 44/300
3456/3456 [==============================] - 0s 64us/step - loss: 4.9927 - acc: 0.7329 - val_loss: 5.2350 - val_acc: 0.7025
Epoch 45/300
3456/3456 [==============================] - 0s 63us/step - loss: 4.9907 - acc: 0.7274 - val_loss: 5.2504 - val_acc: 0.6759
Epoch 46/300
3456/3456 [==============================] - 0s 63us/step - loss: 4.9932 - acc: 0.7170 - val_loss: 5.2368 - val_acc: 0.7315
Epoch 47/300
3456/3456 [==============================] - 0s 68us/step - loss: 4.9925 - acc: 0.7242 - val_loss: 5.2584 - val_acc: 0.6667
Epoch 48/300
3456/3456 [==============================] - 0s 65us/step - loss: 4.9906 - acc: 0.7274 - val_loss: 5.2610 - val_acc: 0.6574
Epoch 49/300
3456/3456 [==============================] - 0s 64us/step - loss: 4.9875 - acc: 0.7280 - val_loss: 5.2459 - val_acc: 0.6829
Epoch 50/300
3456/3456 [==============================

Epoch 103/300
3456/3456 [==============================] - 0s 69us/step - loss: 4.9454 - acc: 0.7370 - val_loss: 5.2907 - val_acc: 0.6713
Epoch 104/300
3456/3456 [==============================] - 0s 69us/step - loss: 4.9480 - acc: 0.7436 - val_loss: 5.3173 - val_acc: 0.7095
Epoch 105/300
3456/3456 [==============================] - 0s 64us/step - loss: 4.9423 - acc: 0.7454 - val_loss: 5.3370 - val_acc: 0.6701
Epoch 106/300
3456/3456 [==============================] - 0s 65us/step - loss: 4.9408 - acc: 0.7477 - val_loss: 5.3182 - val_acc: 0.6887
Epoch 107/300
3456/3456 [==============================] - 0s 69us/step - loss: 4.9435 - acc: 0.7405 - val_loss: 5.3152 - val_acc: 0.6979
Epoch 108/300
3456/3456 [==============================] - 0s 64us/step - loss: 4.9439 - acc: 0.7416 - val_loss: 5.3352 - val_acc: 0.6968
Epoch 109/300
3456/3456 [==============================] - 0s 65us/step - loss: 4.9423 - acc: 0.7413 - val_loss: 5.3568 - val_acc: 0.6956
Epoch 110/300
3456/3456 [=========

3456/3456 [==============================] - 0s 80us/step - loss: 4.9241 - acc: 0.7480 - val_loss: 5.3497 - val_acc: 0.6725
Epoch 163/300
3456/3456 [==============================] - 0s 78us/step - loss: 4.9242 - acc: 0.7558 - val_loss: 5.4067 - val_acc: 0.7002
Epoch 164/300
3456/3456 [==============================] - 0s 67us/step - loss: 4.9224 - acc: 0.7494 - val_loss: 5.3956 - val_acc: 0.6678
Epoch 165/300
3456/3456 [==============================] - 0s 68us/step - loss: 4.9215 - acc: 0.7616 - val_loss: 5.3685 - val_acc: 0.6678
Epoch 166/300
3456/3456 [==============================] - 0s 68us/step - loss: 4.9231 - acc: 0.7543 - val_loss: 5.3708 - val_acc: 0.6898
Epoch 167/300
3456/3456 [==============================] - 0s 66us/step - loss: 4.9212 - acc: 0.7581 - val_loss: 5.4006 - val_acc: 0.6921
Epoch 168/300
3456/3456 [==============================] - 0s 69us/step - loss: 4.9248 - acc: 0.7488 - val_loss: 5.3960 - val_acc: 0.6991
Epoch 169/300
3456/3456 [=======================

3456/3456 [==============================] - 0s 69us/step - loss: 4.9084 - acc: 0.7685 - val_loss: 5.4111 - val_acc: 0.6840
Epoch 222/300
3456/3456 [==============================] - 0s 65us/step - loss: 4.9100 - acc: 0.7749 - val_loss: 5.4141 - val_acc: 0.6979
Epoch 223/300
3456/3456 [==============================] - 0s 66us/step - loss: 4.9167 - acc: 0.7708 - val_loss: 5.4111 - val_acc: 0.6979
Epoch 224/300
3456/3456 [==============================] - 0s 63us/step - loss: 4.9154 - acc: 0.7543 - val_loss: 5.4115 - val_acc: 0.7164
Epoch 225/300
3456/3456 [==============================] - 0s 66us/step - loss: 4.9096 - acc: 0.7766 - val_loss: 5.4238 - val_acc: 0.6794
Epoch 226/300
3456/3456 [==============================] - 0s 66us/step - loss: 4.9140 - acc: 0.7662 - val_loss: 5.4175 - val_acc: 0.7002
Epoch 227/300
3456/3456 [==============================] - 0s 66us/step - loss: 4.9077 - acc: 0.7688 - val_loss: 5.4378 - val_acc: 0.7037
Epoch 228/300
3456/3456 [=======================

3456/3456 [==============================] - 0s 64us/step - loss: 4.9110 - acc: 0.7755 - val_loss: 5.4210 - val_acc: 0.7014
Epoch 281/300
3456/3456 [==============================] - 0s 65us/step - loss: 4.9026 - acc: 0.7815 - val_loss: 5.4102 - val_acc: 0.6736
Epoch 282/300
3456/3456 [==============================] - 0s 63us/step - loss: 4.9032 - acc: 0.7812 - val_loss: 5.4207 - val_acc: 0.6898
Epoch 283/300
3456/3456 [==============================] - 0s 63us/step - loss: 4.9033 - acc: 0.7812 - val_loss: 5.4310 - val_acc: 0.6944
Epoch 284/300
3456/3456 [==============================] - 0s 64us/step - loss: 4.9023 - acc: 0.7755 - val_loss: 5.4300 - val_acc: 0.6933
Epoch 285/300
3456/3456 [==============================] - 0s 82us/step - loss: 4.9045 - acc: 0.7772 - val_loss: 5.4434 - val_acc: 0.7049
Epoch 286/300
3456/3456 [==============================] - 0s 68us/step - loss: 4.9009 - acc: 0.7703 - val_loss: 5.4242 - val_acc: 0.6597
Epoch 287/300
3456/3456 [=======================